### Processing SVI Census tract level data and HepVu death rates data into one dataframe
This is done for all the census tracts in the US

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#import xlsx file as a dataframe
HepVu_df = pd.read_excel('./data/HepVu_County_Opioid_Indicators_05DEC22.xlsx')

In [3]:
HepVu_df.head()

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
0,1001,AL,Autauga County,98.3,8.2,8.8,10.7,9.8,10.9,9.2,11.6
1,1003,AL,Baldwin County,65.0,18.0,20.0,16.6,15.1,14.9,14.5,27.4
2,1005,AL,Barbour County,22.8,4.4,4.5,5.7,5.8,5.2,5.7,7.6
3,1007,AL,Bibb County,24.8,17.2,16.6,22.6,21.7,23.1,19.4,27.3
4,1009,AL,Blount County,22.8,18.6,18.9,22.7,27.0,19.9,20.3,24.2


In [4]:
HepVu_df.shape

(3221, 11)

#### Updating HepVu dataframe "GEO ID" column

###### GEOID column needs to change: insert "0" infront of 4 character values to match other dataframe

In [5]:
HepVu_df["GEO ID"] = HepVu_df["GEO ID"].astype(str)

In [6]:
# Check the length of each value in the 'FIPS' column, and if it's 4 characters, prepend '0'
HepVu_df['GEO ID'] = HepVu_df['GEO ID'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)

In [7]:
HepVu_df["GEO ID"][123]

'05023'

#### Updating SVI 2018 dataframe

In [10]:
# SVI 2018 dataset is in csv format(Issue with downloading as geo file at the moment, so using csv file for now)
SVI_2018_df = gpd.read_file('./data/SVI data/census tract level/SVI2018_US_tract.gdb')

In [20]:
SVI_2018_df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area,geometry
0,01,ALABAMA,AL,01015,Calhoun,01015981901,"Census Tract 9819.01, Calhoun County, Alabama",24.243865,0,12,...,-999,-999,0,12,-999.0,-999.0,656,0.339400,0.006105,"MULTIPOLYGON (((-86.01323 33.68426, -86.01224 ..."
1,01,ALABAMA,AL,01015,Calhoun,01015981902,"Census Tract 9819.02, Calhoun County, Alabama",34.646714,0,12,...,-999,-999,0,12,-999.0,-999.0,146,0.420459,0.008752,"MULTIPOLYGON (((-86.01730 33.69662, -86.01704 ..."
2,01,ALABAMA,AL,01015,Calhoun,01015981903,"Census Tract 9819.03, Calhoun County, Alabama",13.926276,0,12,...,-999,-999,0,12,-999.0,-999.0,0,0.330164,0.003508,"MULTIPOLYGON (((-85.78501 33.66554, -85.78318 ..."


Now we need to add narcotic death rates from HepVu to SVI dataframe \
For each census tract, related county death rates will be added for further computation.

In [17]:
SVI_2018_df['FIPS'][0]

'01015981901'

In [24]:
HepVu_df["GEO ID"][0]

'01001'

In [25]:
SVI_HepVu_2018_df = SVI_2018_df.merge(HepVu_df[['GEO ID','Narcotic Overdose Mortality Rate 2018']], left_on="STCNTY", right_on="GEO ID")

In [26]:
SVI_HepVu_2018_df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area,geometry,GEO ID,Narcotic Overdose Mortality Rate 2018
0,01,ALABAMA,AL,01015,Calhoun,01015981901,"Census Tract 9819.01, Calhoun County, Alabama",24.243865,0,12,...,0,12,-999.0,-999.0,656,0.339400,0.006105,"MULTIPOLYGON (((-86.01323 33.68426, -86.01224 ...",01015,15.8
1,01,ALABAMA,AL,01015,Calhoun,01015981902,"Census Tract 9819.02, Calhoun County, Alabama",34.646714,0,12,...,0,12,-999.0,-999.0,146,0.420459,0.008752,"MULTIPOLYGON (((-86.01730 33.69662, -86.01704 ...",01015,15.8
2,01,ALABAMA,AL,01015,Calhoun,01015981903,"Census Tract 9819.03, Calhoun County, Alabama",13.926276,0,12,...,0,12,-999.0,-999.0,0,0.330164,0.003508,"MULTIPOLYGON (((-85.78501 33.66554, -85.78318 ...",01015,15.8


In [27]:
#get column names to a list
column_names = SVI_HepVu_2018_df.columns.tolist()
column_names

['ST',
 'STATE',
 'ST_ABBR',
 'COUNTY',
 'FIPS',
 'LOCATION',
 'AREA_SQMI',
 'E_TOTPOP',
 'M_TOTPOP',
 'E_HU',
 'M_HU',
 'E_HH',
 'M_HH',
 'E_POV',
 'M_POV',
 'E_UNEMP',
 'M_UNEMP',
 'E_PCI',
 'M_PCI',
 'E_NOHSDP',
 'M_NOHSDP',
 'E_AGE65',
 'M_AGE65',
 'E_AGE17',
 'M_AGE17',
 'E_DISABL',
 'M_DISABL',
 'E_SNGPNT',
 'M_SNGPNT',
 'E_MINRTY',
 'M_MINRTY',
 'E_LIMENG',
 'M_LIMENG',
 'E_MUNIT',
 'M_MUNIT',
 'E_MOBILE',
 'M_MOBILE',
 'E_CROWD',
 'M_CROWD',
 'E_NOVEH',
 'M_NOVEH',
 'E_GROUPQ',
 'M_GROUPQ',
 'EP_POV',
 'MP_POV',
 'EP_UNEMP',
 'MP_UNEMP',
 'EP_PCI',
 'MP_PCI',
 'EP_NOHSDP',
 'MP_NOHSDP',
 'EP_AGE65',
 'MP_AGE65',
 'EP_AGE17',
 'MP_AGE17',
 'EP_DISABL',
 'MP_DISABL',
 'EP_SNGPNT',
 'MP_SNGPNT',
 'EP_MINRTY',
 'MP_MINRTY',
 'EP_LIMENG',
 'MP_LIMENG',
 'EP_MUNIT',
 'MP_MUNIT',
 'EP_MOBILE',
 'MP_MOBILE',
 'EP_CROWD',
 'MP_CROWD',
 'EP_NOVEH',
 'MP_NOVEH',
 'EP_GROUPQ',
 'MP_GROUPQ',
 'EPL_POV',
 'EPL_UNEMP',
 'EPL_PCI',
 'EPL_NOHSDP',
 'SPL_THEME1',
 'RPL_THEME1',
 'EPL_AGE65',
 'E

In [34]:
# rename  'Opioid Prescription Rate 2018'
SVI_HepVu_2018_df.rename(columns = {'Narcotic Overdose Mortality Rate 2018':'NOD_Rate'}, inplace = True)

In [35]:
# Convert the DataFrame to a GeoDataFrame
SVI_HepVu_2018_df = gpd.GeoDataFrame(SVI_HepVu_2018_df, geometry='geometry')

In [36]:
# save SVI_HepVu_2018_df as a shapefile
SVI_HepVu_2018_df.to_file("./data/processed data/SVI 2018 with HepVu/SVI2018_US_census_with_opioid_indicators.shp")